In [118]:

from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from operator import itemgetter
from dotenv import load_dotenv
load_dotenv('/Users/eshantdas/Desktop/Youtube/Youtube_GenAI_Test_code/myenv/.env')
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

model = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
)

#### Problems without tools

In [119]:
prompt = ChatPromptTemplate.from_template("{content}")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

answer=chain.invoke({"content": "What is (4.5*2.1)**2.2 "})
print(answer)

To evaluate this expression, we need to follow the order of operations (PEMDAS):

1. Multiply 4.5 and 2.1: 4.5*2.1 = 9.45
2. Raise 9.45 to the power of 2.2: 9.45**2.2 ≈ 102.34

So, (4.5*2.1)**2.2 ≈ 102.34.


#### How to use Tool

In [120]:

from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from operator import itemgetter

In [121]:
#Using in built math tool

tool=load_tools(['llm-math'],llm=model)
math_tool=tool[0]
math_tool

Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=ChatGroq(client=<groq.resources.chat.completions.Completio

In [122]:
math_tool.run("What is 2+2")

'Answer: 4'

In [123]:
math_tool.run("What is (4.5*2.1)**2.2")

'Answer: 139.94261298333066'

In [124]:
(4.5*2.1)**2.2

139.94261298333066

In [125]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [126]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [127]:
wiki_tool

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/eshantdas/Desktop/Youtube/Youtube_GenAI_Test_code/myenv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=1000))

In [128]:
answer=wiki_tool.run({"query": "Rohit Sharma"})
print(answer)

Page: Rohit Sharma
Summary: Rohit Gurunath Sharma (born 30 April 1987) is an Indian international cricketer who currently plays for and captains the India national cricket team in Test and One Day International (ODI) matches. Previously, he also captained the team in Twenty20 International (T20I) matches and led India's win in 2024 ICC Men's T20 World Cup, subsequent to which he retired from T20s in June, 2024. The right-handed batsman, who is considered one of the best batsmen of his generation and one of the greatest opening batters of all time, is known for his timing, elegance, six-hitting abilities and leadership skills.
He holds several batting records which famously include most runs in T20 Internationals, most sixes in international cricket, most double centuries in ODI cricket (3), most centuries at Cricket World Cups (7) and joint most hundreds in Twenty20 Internationals (5). He is the first player to score 5 T20I centuries. He also holds the world record for the highest indi

## Agents

In [129]:
tools=[math_tool,wiki_tool]

In [130]:
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=ChatGroq(client=<groq.resources.chat.completions.Completi

In [131]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [132]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [139]:
# Construct the tool calling agent
agent = create_tool_calling_agent(model.bind_tools(tools), tools, prompt)

In [140]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,return_intermediate_steps=True,handle_parsing_errors=True)

In [141]:
agent_executor.invoke(
    {
        "input": "What is (4.5*2.1)**2.2 "
    }
)



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `(4.5*2.1)**2.2`


Answer: 139.94261298333066The answer is 139.94261298333066.

> Finished chain.


{'input': 'What is (4.5*2.1)**2.2 ',
 'output': 'The answer is 139.94261298333066.',
 'intermediate_steps': [(ToolAgentAction(tool='Calculator', tool_input='(4.5*2.1)**2.2', log='\nInvoking: `Calculator` with `(4.5*2.1)**2.2`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'id': 'call_e320', 'function': {'arguments': '{"__arg1":"(4.5*2.1)**2.2"}', 'name': 'Calculator'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a5e2427a-2f1d-47e0-8ade-e64f2d9df526', tool_calls=[{'name': 'Calculator', 'args': {'__arg1': '(4.5*2.1)**2.2'}, 'id': 'call_e320', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1075, 'output_tokens': 54, 'total_tokens': 1129}, tool_call_chunks=[{'name': 'Calculator', 'args': '{"__arg1":"(4.5*2.1)**2.2"}', 'id': 'call_e320', 'index': None, 'type': 'tool_call_chunk'}])], tool_call_id='call_e320'),
   'Answer: 139.94261298333066')]}

In [142]:
agent_executor.invoke(
    {
        "input": "When was ROhit Sharma born"
    }
)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Rohit Sharma'}`


Page: Rohit Sharma
Summary: Rohit Gurunath Sharma (born 30 April 1987) is an Indian international cricketer who currently plays for and captains the India national cricket team in Test and One Day International (ODI) matches. Previously, he also captained the team in Twenty20 International (T20I) matches and led India's win in 2024 ICC Men's T20 World Cup, subsequent to which he retired from T20s in June, 2024. The right-handed batsman, who is considered one of the best batsmen of his generation and one of the greatest opening batters of all time, is known for his timing, elegance, six-hitting abilities and leadership skills.
He holds several batting records which famously include most runs in T20 Internationals, most sixes in international cricket, most double centuries in ODI cricket (3), most centuries at Cricket World Cups (7) and joint most hundreds in Twenty20 Internationals (5). He 

{'input': 'When was ROhit Sharma born',
 'output': 'Rohit Sharma was born on 30 April 1987.',
 'intermediate_steps': [(ToolAgentAction(tool='wikipedia', tool_input={'query': 'Rohit Sharma'}, log="\nInvoking: `wikipedia` with `{'query': 'Rohit Sharma'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'id': 'call_xj01', 'function': {'arguments': '{"query":"Rohit Sharma"}', 'name': 'wikipedia'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4b0c9123-a7b2-49aa-9f74-3754e4025e8d', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'Rohit Sharma'}, 'id': 'call_xj01', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1066, 'output_tokens': 46, 'total_tokens': 1112}, tool_call_chunks=[{'name': 'wikipedia', 'args': '{"query":"Rohit Sharma"}', 'id': 'call_xj01', 'index': None, 'type': 'tool_call_chunk'}])], tool_call_id='call_xj01'),
   "Page: Rohit Sharma\nSummary: Rohit Gurunath Sharma (born 30 April

In [143]:
a=agent_executor.invoke(
    {
        "input": "Describe  Rohit Sharma"
    }
)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Rohit Sharma'}`


Page: Rohit Sharma
Summary: Rohit Gurunath Sharma (born 30 April 1987) is an Indian international cricketer who currently plays for and captains the India national cricket team in Test and One Day International (ODI) matches. Previously, he also captained the team in Twenty20 International (T20I) matches and led India's win in 2024 ICC Men's T20 World Cup, subsequent to which he retired from T20s in June, 2024. The right-handed batsman, who is considered one of the best batsmen of his generation and one of the greatest opening batters of all time, is known for his timing, elegance, six-hitting abilities and leadership skills.
He holds several batting records which famously include most runs in T20 Internationals, most sixes in international cricket, most double centuries in ODI cricket (3), most centuries at Cricket World Cups (7) and joint most hundreds in Twenty20 Internationals (5). He 

ValueError: unknown format from LLM: I think there might be some mistake! The question "Rohit Sharma is a renowned Indian cricketer" doesn't seem to be a math problem. It's a statement about a person.

If you meant to ask a math-related question, please feel free to rephrase or provide a new question. I'd be happy to help!

In [ ]:
a

{'input': 'Describe  Rohit Sharma',
 'output': 'Rohit Sharma is an Indian international cricketer who currently plays for and captains the India national cricket team in Test and One Day International (ODI) matches. He is a right-handed batsman known for his timing, elegance, and six-hitting abilities, and is considered one of the best batsmen of his generation and one of the greatest opening batters of all time.',
 'intermediate_steps': [(ToolAgentAction(tool='wikipedia', tool_input={'query': 'Rohit Sharma'}, log="\nInvoking: `wikipedia` with `{'query': 'Rohit Sharma'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'id': 'call_qb4w', 'function': {'arguments': '{"query":"Rohit Sharma"}', 'name': 'wikipedia'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a8561ff7-5965-4baf-9a60-47ec30583fff', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'Rohit Sharma'}, 'id': 'call_qb4w', 'type': 'tool_cal

## Creating custom tools

In [ ]:
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def extract_first_word(query: str) -> str:
    """EXTRACT THE FIRST WORD FROM A SENTENCE"""
    return query.split()[0]

In [ ]:
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def llm_call(query: str) -> str:
    """IF any compliecated question or logical reasoning comes which other tools cant then we use this for an llm response"""
    prompt = ChatPromptTemplate.from_template("{content}")
    output_parser = StrOutputParser()

    chain = prompt | model | output_parser

    answer=chain.invoke({"content": query})
    return answer
    

In [ ]:
extract_first_word.run("Eshant is happy")

'Eshant'

In [ ]:
tools.append(llm_call)

In [ ]:
# Construct the tool calling agent
agent = create_tool_calling_agent(model, tools, prompt)

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,return_intermediate_steps=True,handle_parsing_errors=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "Give me logical reasoning questions"
    }
)



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `logical reasoning questions`




ValueError: unknown format from LLM: I'm happy to help! However, I notice that the last question is a "logical reasoning question" which doesn't seem to be a math problem that can be translated into a Python expression. Could you please provide more context or clarify what you mean by "logical reasoning question"? Are you looking for help with a specific problem or would you like me to assist with a different math problem?

If you meant to ask a different math question, feel free to provide it, and I'll be happy to help!

In [ ]:
from langchain.chains import LLMMathChain, LLMChain